# Draft Assistant

A notebook-based draft assistant.

In [1]:
# Install package
%pip install .. -q

import pandas as pd
import statisticaldrafting as sd

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Load a model. 
# sd.list_sets() # Show available sets.
dm = sd.DraftModel(set="FDN", draft_mode="Premier")

In [ ]:
# collection = ["Vivien Reid", "Scavenging Ooze"] # Collection as cardnames
# collection = [16, 229, 78, 78, 198] # Collection as card ids
# collection = [174, 74, 94, 233, 266, 239, 134, 171, 23] * 1 # + ["Soulstone Sanctuary"] * 1
# collection = [66] * 5 + [134] * 5 + [188] * 2 + [169] * 4 + [129] * 4 + [100] * 3 + [214] * 4 # BW FDN deck
collection = []
pick_order = dm.get_pick_order(collection)
# pick_order = pick_order[pick_order["rarity"].isin(["common", "uncommon"])] # Optional filter. 
# pick_order = pick_order[pick_order["color_identity"].isin(["R"])] # Optional filter. 
pick_order

,name,rarity,color_identity,p1p1_rating,synergy,rating
63,Curator of Destinies,rare,U,95.2,2.5,97.7
162,"Liliana, Dreadhorde General",mythic,B,95.7,1.8,97.5
35,Bloodthirsty Conqueror,mythic,B,92.2,4.8,97.0
223,Sire of Seven Deaths,mythic,Colorless,91.3,5.2,96.5
174,Mischievous Mystic,uncommon,U,88.5,7.3,95.8
155,"Kiora, the Rising Tide",rare,U,92.2,3.2,95.4
74,Dreadwing Scavenger,uncommon,Multicolor,81.4,13.2,94.6
33,Blasphemous Edict,rare,B,87.5,6.7,94.2
94,Faebloom Trick,uncommon,U,84.2,9.0,93.2
138,High-Society Hunter,rare,B,91.3,1.8,93.1


In [4]:
# Deckbuild recommendations after draft. 
# dm.get_deck_recommendation(coll]"ection, starting_colors="").head(30)